In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

# Load MyanBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("UCSYNLP/MyanBERTa")

# Example Myanmar text
text = "မြန်မာဘာသာစကားကို AI နှင့် 처리ပါ"

# Tokenize text



/home/yewinnaing/miniconda3/envs/my_chatbot_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_data = [
    {"text": "ဈေးနှုန်းဘယ်လောက်လဲ?", "label": 1},  # (Wants to buy)
    {"text": "ဘယ်မှာ ဝယ်လို့ရမလဲ?", "label": 1},  # (Wants to buy)
    {"text": "မလို့မဝယ်သေးဘူး", "label": 0},  # (Not buying)
    {"text": "စျေးကြည့်တာပဲ", "label": 0}  # (Not buying)
]


In [4]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [5]:
dataset = Dataset.from_list(train_data)
dataset = dataset.map(tokenize_function, batched=True)

# Load model (binary classification)
model = AutoModelForSequenceClassification.from_pretrained("UCSYNLP/MyanBERTa", num_labels=2)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 4/4 [00:00<00:00, 22.58 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at UCSYNLP/MyanBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs"
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)


In [8]:
trainer.train()

ValueError: expected sequence of length 10 at dim 1 (got 12)

In [ ]:
def predict_intent(text):
    tokens = tokenizer(text, return_tensors="pt")
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()
    return "Wants to buy" if prediction == 1 else "Not interested"

In [ ]:
print(predict_intent("ဈေးနှုန်းဘယ်လောက်လဲ?"))  # Output: Wants to buy
print(predict_intent("စျေးကြည့်တာပဲ"))  # Outpu